# Demo for testing each stage

Please feel free to change q1 and q2 in the second block to test each clause. 

In [1]:
from z3 import *
from bool_test import QueryTest
from test_info import *
from utils import *
import time


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [2]:

q1 = """
SELECT t2.author
FROM inproceedings t1, authorship t2, inproceedings t3, authorship t4
WHERE t1.yearx = t3.yearx
AND t1.area <> t3.area
AND t1.yearx <> 2015 
GROUP BY t2.author, t1.booktitle
"""

q2 = """
SELECT t2.author
FROM inproceedings t1, authorship t2, inproceedings t3, authorship t4
WHERE t1.area <> t3.area 
AND t1.yearx < 2015 OR t1.yearx > 2015
AND t1.yearx = t3.yearx
GROUP BY t2.author, t1.area
"""


# q1 = """
# SELECT DISTINCT t2.author
# FROM inproceedings t1, authorship t2, inproceedings t3, authorship t4
# WHERE (t1.yearx = 1
# AND t3.yearx = 2) 
# OR (t1.area = 'Database'
# AND t3.area = 'ML-AI')
# """

# q2 = """
# SELECT t2.author
# FROM inproceedings t1, authorship t2, inproceedings t3, authorship t4
# WHERE (t1.yearx = 4
# AND (t3.yearx = 5 OR (t1.area = 'Database'
# AND t3.area = 'ML-AI')) ) 
# OR (t1.area = 'Database'
# AND t3.area = 'ML-AI')
# GROUP BY t2.author
# """


q1_info = QueryInfo(q1)
q2_info = QueryInfo(q2)

In [3]:
m = MappingInfo(q1_info, q2_info)

In [4]:
print(len(m.table_mappings))

4


In [5]:
t = QueryTest(q1_info, q2_info, m.z3_var_lookup, m.table_mappings[0], m.table_mappings_reverse[0])

In [6]:
# This block test Qr-Hint optimized
start = time.time()
# t.test_where_having_min_overall_fg() # explore all repair sites and return the smallest fixes
t.test_where_having_min_rs_ttg() # use the first valid repair sites and fixes
end = time.time()
print(end-start)
# t.test_where_having_min_overall_ttg()

Number of sets of disjoint trees to explore:  28
********* New Iteration *********
# of necessary predicates: 5
# of repair sites: 2
Repair Site #0: t1.area != t3.area
Repair Site size #0: 1
Fix #0: False
Fix Size #0: 1
Repair Site #1: t1.yearx > 2015
Repair Site size #1: 1
Fix #1: (t1.area != t3.area AND t1.yearx != 2015)
Fix Size #1: 3
2.245651960372925


In [6]:
# This block test Qr-Hint
start = time.time()
# t.test_where_having_min_overall_fg()  # explore all repair sites
t.test_where_having_min_rs_fg() # use the first valid repair sites
end = time.time()
print(end-start)

iter len:  28
Repair Site #0: t1.area != t3.area
Repair Site size #0: 1
Fix #0: t1.yearx == t3.yearx AND t1.area != t3.area
Fix Size #0: 3
Repair Site #1: t1.yearx > 2015
Repair Site size #1: 1
Fix #1: t1.area != t3.area AND t1.yearx != 2015
Fix Size #1: 3
1.9335548877716064


In [6]:
# test group by
incorrect, missing = t.test_group_by()
print("Incorrect: ", incorrect)
print("Missing: ", missing)

Incorrect:  ['t1.area']
Missing:  ['t1.booktitle']


In [7]:
# test select
incorrect, wrong_position, missing = t.test_select()
print("Incorrect: ", incorrect)
print("Correct but wrong position: ", wrong_position)
print("Missing: ", missing)

([], [])